In [13]:
# !pip install trectools
import pandas as pd
import trectools as trec

In [14]:
results_df = pd.read_csv("./MQ2007-agg/result_agg_functions_MQ2007_top21.csv", sep="\t")
results_df.head(5)

,qid,docid,relevance,ranking_1,ranking_2,ranking_3,ranking_4,ranking_5,ranking_6,ranking_7,...,ranking_15_rank,ranking_16_rank,ranking_17_rank,ranking_18_rank,ranking_19_rank,ranking_20_rank,ranking_21_rank,rrf,squared_rrf,mrrf
0,10,GX000-00-0000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
1,10,GX000-24-12369390,1,107,107,85,0,113,110,110,...,3,8,0,7,6,14,6,0.246088,0.435389,0.840659
2,10,GX000-62-7863450,1,0,0,0,1,8,14,14,...,29,4,0,0,7,0,0,0.160114,0.228318,0.509268
3,10,GX016-48-5543459,1,0,0,111,4,1,3,3,...,26,10,3,1,18,7,1,0.241809,2.211949,0.776428
4,10,GX037-87-3082362,0,160,160,117,57,102,209,209,...,17,7,0,0,13,22,17,0.240359,0.369523,0.804108


In [15]:
qrel_df = results_df[["qid", "docid", "relevance"]].copy()
qrel_df['rank'] = qrel_df.groupby('qid')['relevance'].rank(method='dense', ascending=False).astype(int)
qrel_df['q0']  = 'Q0'
qrel_df['rel'] = qrel_df['relevance']
qrel_df['query'] = qrel_df['qid']
qrel_df = qrel_df[["qid","query", "q0", "docid", "rel"]]
print(qrel_df.head(5))
trec_qrel = trec.TrecQrel()
trec_qrel.qrels_data = qrel_df.copy()

   qid  query  q0              docid  rel
0   10     10  Q0   GX000-00-0000000    0
1   10     10  Q0  GX000-24-12369390    1
2   10     10  Q0   GX000-62-7863450    1
3   10     10  Q0   GX016-48-5543459    1
4   10     10  Q0   GX037-87-3082362    0


In [16]:
# Evaluate RRF
run_df_rrf = results_df[["qid", "docid", "rrf"]].copy()
run_df_rrf['rank'] = run_df_rrf.groupby('qid')['rrf'].rank(method='dense', ascending=False).astype(int)
run_df_rrf['q0']  = 'Q0'
run_df_rrf['score'] = run_df_rrf['rrf']
run_df_rrf['system'] = 'rrf'
run_df_rrf['query'] = run_df_rrf['qid']
run_df_rrf = run_df_rrf[["qid","query","q0", "docid","rank" ,"score", "system"]]
# print(run_df_rrf.head(5))

# RRF Evaluation
run_rrf = trec.TrecRun()
run_rrf.run_data = run_df_rrf.copy()
rrf_trec_eval = trec.TrecEval(run_rrf, trec_qrel)
print(f"rrd:{rrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    print(f"ndcg@{i}:{rrf_trec_eval.get_ndcg(depth=i, trec_eval=True)}")
    print(f"precision@{i}:{rrf_trec_eval.get_precision(depth=i, trec_eval=True)}")
    print(f"map@{i}:{rrf_trec_eval.get_map(depth=i, trec_eval=True)}")
    


rrd:17991
ndcg@1:0.11495271867612293
precision@1:0.29314420803782504
map@1:0.024331686732735055
ndcg@2:0.1543492765042202
precision@2:0.2990543735224586
map@2:0.04327102376003841
ndcg@3:0.17705174781133343
precision@3:0.30752561071710005
map@3:0.06039610848945622
ndcg@4:0.19128190614698964
precision@4:0.30939716312056736
map@4:0.07577169797887552
ndcg@5:0.20661125636608038
precision@5:0.308628841607565
map@5:0.08886388870597921
ndcg@6:0.21945068872951817
precision@6:0.31067769897557135
map@6:0.10259290927283486
ndcg@7:0.22962779244624548
precision@7:0.3116345829111786
map@7:0.11521123526201127
ndcg@8:0.23913365898902117
precision@8:0.3108747044917258
map@8:0.12657853822096038
ndcg@9:0.2483270977285938
precision@9:0.310086682427108
map@9:0.1377452814575264
ndcg@10:0.2569554091776554
precision@10:0.3105791962174941
map@10:0.14865819492472573


In [17]:
# Evaluate SRRF
run_df_srrf = results_df[["qid", "docid", "squared_rrf"]].copy()
run_df_srrf['rank'] = run_df_srrf.groupby('qid')['squared_rrf'].rank(method='dense', ascending=False).astype(int)
run_df_srrf['q0']  = 'Q0'
run_df_srrf['score'] = run_df_srrf['squared_rrf']
run_df_srrf['system'] = 'srrf'
run_df_srrf['query'] = run_df_srrf['qid']
run_df_srrf = run_df_srrf[["qid","query","q0", "docid","rank" ,"score", "system"]]
# print(run_df_srrf.head(5))

# SRRF Evaluation
run_srrf = trec.TrecRun()
run_srrf.run_data = run_df_srrf.copy()
srrf_trec_eval = trec.TrecEval(run_srrf, trec_qrel)
print(f"rrd:{srrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    # print(f"ndcg@{i}:{srrf_trec_eval.get_ndcg(depth=i)}")
    print(f"ndcg@{i}:{srrf_trec_eval.get_ndcg(depth=i, trec_eval=True)}")
    print(f"precision@{i}:{srrf_trec_eval.get_precision(depth=i, trec_eval=True)}")
    print(f"map@{i}:{srrf_trec_eval.get_map(depth=i, trec_eval=True)}")
    

rrd:17991
ndcg@1:0.11495271867612293
precision@1:0.19267139479905437
map@1:0.015175676826839781
ndcg@2:0.1543492765042202
precision@2:0.20124113475177305
map@2:0.025389858154483018
ndcg@3:0.17705174781133343
precision@3:0.2052797478329393
map@3:0.03416587994476849
ndcg@4:0.19128190614698964
precision@4:0.20966312056737588
map@4:0.04215164668420609
ndcg@5:0.20661125636608038
precision@5:0.21347517730496457
map@5:0.04977507593925829
ndcg@6:0.21945068872951817
precision@6:0.2183806146572104
map@6:0.05709897336338148
ndcg@7:0.22962779244624548
precision@7:0.22019587977034785
map@7:0.06397953235941148
ndcg@8:0.23913365898902117
precision@8:0.22421690307328604
map@8:0.07126780878573939
ndcg@9:0.2483270977285938
precision@9:0.22951142631993698
map@9:0.0789532427915186
ndcg@10:0.2569554091776554
precision@10:0.23268321513002366
map@10:0.08640067154031061


In [18]:
# Evaluate MRRF
run_df_mrrf = results_df[["qid", "docid", "mrrf"]].copy()
run_df_mrrf['rank'] = run_df_mrrf.groupby('qid')['mrrf'].rank(method='dense', ascending=False).astype(int)
run_df_mrrf['q0']  = 'Q0'
run_df_mrrf['score'] = run_df_mrrf['mrrf']
run_df_mrrf['system'] = 'mrrf'
run_df_mrrf['query'] = run_df_mrrf['qid']
run_df_mrrf = run_df_mrrf[["qid","query","q0", "docid","rank", "score", "system"]]
# print(run_df_mrrf.head(5))

# MRRF Evaluation
run_mrrf = trec.TrecRun()
run_mrrf.run_data = run_df_mrrf.copy()
mrrf_trec_eval = trec.TrecEval(run_mrrf, trec_qrel)
print(f"rrd:{mrrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    print(f"ndcg@{i}:{mrrf_trec_eval.get_precision(depth=i, trec_eval=True)}")
    print(f"precision@{i}:{mrrf_trec_eval.get_precision(depth=i, trec_eval=True)}")
    print(f"map@{i}:{mrrf_trec_eval.get_map(depth=i, trec_eval=True)}")
    

rrd:17991
ndcg@1:0.26832151300236406
precision@1:0.26832151300236406
map@1:0.02107118705243639
ndcg@2:0.2718676122931442
precision@2:0.2718676122931442
map@2:0.03713903265785619
ndcg@3:0.27876280535855
precision@3:0.27876280535855
map@3:0.051375756364378355
ndcg@4:0.28028959810874704
precision@4:0.28028959810874704
map@4:0.06422708831948715
ndcg@5:0.2842789598108747
precision@5:0.2842789598108747
map@5:0.07659491654753603
ndcg@6:0.2857565011820331
precision@6:0.2857565011820331
map@6:0.0881624996701983
ndcg@7:0.2885849375211077
precision@7:0.2885849375211077
map@7:0.09958258640991578
ndcg@8:0.28937647754137114
precision@8:0.28937647754137114
map@8:0.11023320956669003
ndcg@9:0.2907801418439716
precision@9:0.2907801418439716
map@9:0.12076681535047072
ndcg@10:0.2895390070921986
precision@10:0.2895390070921986
map@10:0.13016797794306564
